In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt

In [2]:
classes = {
    'c0' : "safe_driving",
    'c1' : "texting-right",
    'c2' : "talking_on_the_phone-right",
    'c3' : "texting-left",
    'c4' : "talking_on_the_phone-left",
    'c5' : "operating_the_radio",
    'c6' : "drinking",
    'c7' : "reaching_behind",
    'c8' : "hair-and-makeup",
    'c9' : "talking_to_passenger"
}

In [3]:
train_dataset_filepath = "../input/state-farm-distracted-driver-detection/imgs/train"
test_dataset_filepath = "../input/state-farm-distracted-driver-detection/imgs/test"

In [4]:
for detection_class in os.listdir(train_dataset_filepath):
    path_to_img_dir = train_dataset_filepath + "/" + detection_class
    print(detection_class + " has " + str(len(os.listdir(path_to_img_dir))) + " images.")

In [5]:
plt.figure(figsize = (12, 20))
img_count = 1

for detection_class in os.listdir(train_dataset_filepath):
    path_to_img_dir = train_dataset_filepath + "/" + detection_class
    fig = plt.subplot(5, 2, img_count)
    img_count += 1
    sample_img_path = path_to_img_dir + "/" + random.choice(os.listdir(path_to_img_dir))
    sample_img = cv2.imread(sample_img_path)
    sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
    plt.imshow(sample_img)
    plt.title(detection_class + ":" + classes[detection_class])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications.imagenet_utils import preprocess_input 

img_data_generator = ImageDataGenerator(
    preprocessing_function= resnet50.preprocess_input,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    fill_mode="nearest",
    validation_split=0.1)

In [7]:
batch_size = 32

total_num_of_images = 0
for detection_class in os.listdir(train_dataset_filepath):
    path_to_img_dir = train_dataset_filepath + "/" + detection_class
    total_num_of_images += len(os.listdir(path_to_img_dir))

train_batches = img_data_generator.flow_from_directory(
    directory=train_dataset_filepath,
    shuffle=True,
    batch_size=batch_size,
    subset="training")
val_batches = img_data_generator.flow_from_directory(
    directory=train_dataset_filepath,
    shuffle=True,
    batch_size=batch_size,
    subset="validation")

In [8]:
from tensorflow.keras import layers, models

resnet50_model_for_transfer_learning = resnet50.ResNet50(weights='imagenet',include_top=False,input_shape = (256,256,3))

for layer in resnet50_model_for_transfer_learning.layers[:-3]:
    layer.trainable=False
    
resnet_model = models.Sequential()
resnet_model.add(resnet50_model_for_transfer_learning)
resnet_model.add(layers.Flatten())

resnet_model.add(layers.Dense(512,activation = 'relu',))
resnet_model.add(layers.BatchNormalization())

resnet_model.add(layers.Dense(256,activation = 'relu'))
resnet_model.add(layers.BatchNormalization())

resnet_model.add(layers.Dense(128,activation = 'relu'))
resnet_model.add(layers.BatchNormalization())

resnet_model.add(layers.Dense(10,activation = 'softmax',name = 'output'))

In [9]:
from tensorflow.keras.utils import plot_model

plot_model(resnet_model,"model.png",show_shapes=True,show_layer_names=True)

In [10]:
from tensorflow.keras import optimizers

resnet_model.compile(
    optimizer = optimizers.Adam(learning_rate=.0001),
    loss='categorical_crossentropy',
    metrics=['acc'])

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

stop_criteria = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
resnet_model_history = resnet_model.fit(x=train_batches,
          steps_per_epoch=250,
          epochs=15,
          validation_data=val_batches,
          validation_steps= 50,
          callbacks=[stop_criteria])